## Project: Milestone 4

### DSC 540

#### Taniya Adhikari 2/26/2021

In [1]:
from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import urllib.request, urllib.parse 
from urllib.error import HTTPError,URLError 
import json
from urllib.request import urlopen
import warnings; warnings.simplefilter('ignore')
from time import sleep
from pandas import json_normalize 

%matplotlib inline

### 1. Reading the data from API and Converting it into dataframe

First get the list of ticker symbols from the previous mil.

In [2]:
df = pd.read_csv("clean_htm_data.csv")

In [3]:
ticker_symbols = df["Ticker"].tolist()

In [4]:
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
  
# How many elements each 
# list should have 
n = 5
  
ticker = list(divide_chunks(ticker_symbols, n)) 


Splitting the ticker symbols into multiple chunks, to retrieve data. API has limitations of 5 ticker symbol per request and has only 250 daily limitations.

In [7]:
len(ticker)

445

In [8]:
def get_jsonparsed_data(url):
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [11]:
def retrieve_data(ticker, api, n, i):
    serviceurl = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    time = '?from=2021-02-15&to=2021-02-26&'
    i = i
    batch = ''
    appended_data = []

    while i < n:
        batch = ','.join(ticker[i])
        url = (serviceurl + batch + time + api)
        
        # API terms to wait 10 seconds before requesting next
        sleep(10)
        data = get_jsonparsed_data(url)
        # checks for missing symbols list
        try:
            historical_data = json_normalize(data=data['historicalStockList'], record_path='historical', meta='symbol')
            appended_data.append(historical_data)
        except:
            print('list not found')
            print(batch)
            pass

        i = i+1
        
    df = pd.concat(appended_data)
    return df

Batch 1

In [14]:
api_1 = 'apikey=2d9d208e7f3ddc49049cbdd07b1f1db3'
df_1 = retrieve_data(ticker, api_1, 250, 0)

list not found
MCRI,LGVW,CTS,SIBN,GNMK
list not found
AJAX,BOLT,CHCO,MGPI,ADVM
list not found
CLOV,ITOS,EVER,RDWR,EIGI
list not found
VOR,RPTX,TNC,CEVA,TPGY
list not found
SUPN,WABC,VRTU,NKTX,HEP
list not found
KAI,BANR,GTN.A,PAR,NTB
list not found
ZYME,TWNK,BUR,BH.A,CSWI
list not found
TAC,BEAT,MCRB,PUBM,ANAT
list not found
MOG.A,FUN,TTGT,RKT,FCFS
list not found
MOG.B,ENBL,BRC,PDCE,TRQ
list not found
ATKR,JW.B,UCBI,CVBF,SMPL
list not found
PRMW,WERN,SFNC,NNI,JW.A
list not found
ITCI,MC,LGF.B,BOX,CATY
list not found
BECN,GHC,LAUR,GEF.B,EAF
list not found
HWC,NNOX,LGF.A,CNS,ENR
list not found
BCO,CWEN.A,TRN,OPK,ALKS


Batch 2

In [20]:
api_2 = 'apikey=1c7a6e10642f30c26926b9fe76ff7293'
df_2 = retrieve_data(ticker, api_2, 445, 250)

list not found
SNA,HSIC,SBNY,IVZ,WSO.B
list not found
UHS,RPM,AXON,TAP.A,PEGA
list not found
CGNX,PBA,JBHT,WAB,HEI.A
list not found
CZR,AVTR,BIO.B,HEI,DOV
list not found
CBRE,VRSN,LEN.B,FITB,MGA
list not found
MKC.V,CLX,MKC,YUMC,ANET
list not found
ZBH,BF.A,XEL,VIAC,VIACA
list not found
INFO,MSCI,RACE,IQV,BF.B
list not found
STZ.B,EA,TDOC,KHC,MTCH
list not found
V,BRK.B,BRK.A,FB,TSLA


Now the data is ready to be merged into single file

In [22]:
df_1.head(2)

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2021-02-26,25.150000,26.820000,24.290001,26.040001,26.040001,621381.0,621381.0,0.89,3.539,25.71667,"February 26, 21",0.03539,TGH
1,2021-02-25,27.059999,27.530001,25.180000,25.260000,25.260000,633600.0,633600.0,-1.80,-6.652,25.99000,"February 25, 21",-0.06652,TGH


In [23]:
df_2.head(2)

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2021-02-26,36.80,37.300000,36.185000,36.60,36.60,977381.0,977381.0,-0.20,-0.543,36.695,"February 26, 21",-0.00543,SKX
1,2021-02-25,38.02,38.560001,36.240002,36.68,36.68,991300.0,991300.0,-1.34,-3.524,37.160,"February 25, 21",-0.03524,SKX


In [25]:
api_df = pd.concat([df_1, df_2])
api_df.shape

(18853, 14)

### 2. Stripping string data of whitespace

In [29]:
def no_whitespace(string):
    return string.strip()

# stripping white space from all column type as object
def clean_string(df):
    
    for col in df.columns:
        if df[col].dtype == np.object:
            new = '{}_new'.format(col)
            df[new] = df[col].apply(no_whitespace)
            df[col] = df[new]
            df = df.drop(columns=[new])
    return df


In [32]:
api_df = clean_string(api_df)
api_df.head(5)

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2021-02-26,25.150000,26.820000,24.290001,26.040001,26.040001,621381.0,621381.0,0.89,3.539,25.71667,"February 26, 21",0.03539,TGH
1,2021-02-25,27.059999,27.530001,25.180000,25.260000,25.260000,633600.0,633600.0,-1.80,-6.652,25.99000,"February 25, 21",-0.06652,TGH
2,2021-02-24,28.000000,28.450001,26.770000,26.980000,26.980000,1092300.0,1092300.0,-1.02,-3.643,27.40000,"February 24, 21",-0.03643,TGH
3,2021-02-23,27.400000,29.170000,25.500000,28.820000,28.820000,1088500.0,1088500.0,1.42,5.182,27.83000,"February 23, 21",0.05182,TGH
4,2021-02-22,25.030001,27.980000,24.990000,27.510000,27.510000,1569000.0,1569000.0,2.48,9.908,26.82667,"February 22, 21",0.09908,TGH


### 3. Subsetting or Filtering dataset

Removing repeated columns

In [34]:
api_df.drop(['adjClose', 'unadjustedVolume', 'changeOverTime', 'label'], axis=1, inplace=True)
api_df.shape

(18853, 10)

### 4. Transforming datatype to correct format

In [35]:
# checking data type
def Datatype(df):
    # shape and data types of the data
    print("There are {} rows and {} columns".format(df.shape[0], df.shape[1]))
    print(df.dtypes)

    # select numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    numeric_cols = df_numeric.columns.values
    print(numeric_cols)

    # select non numeric columns
    df_non_numeric = df.select_dtypes(exclude=[np.number])
    non_numeric_cols = df_non_numeric.columns.values
    print(non_numeric_cols)

In [36]:
Datatype(api_df)

There are 18853 rows and 10 columns
date              object
open             float64
high             float64
low              float64
close            float64
volume           float64
change           float64
changePercent    float64
vwap             float64
symbol            object
dtype: object
['open' 'high' 'low' 'close' 'volume' 'change' 'changePercent' 'vwap']
['date' 'symbol']


In [39]:
api_df['date'] = pd.to_datetime(api_df['date'])
Datatype(api_df)

There are 18853 rows and 10 columns
date             datetime64[ns]
open                    float64
high                    float64
low                     float64
close                   float64
volume                  float64
change                  float64
changePercent           float64
vwap                    float64
symbol                   object
dtype: object
['open' 'high' 'low' 'close' 'volume' 'change' 'changePercent' 'vwap']
['date' 'symbol']


### 5. Renaming data headers

In [40]:
# renaming headers
api_df = api_df.rename(columns={"changePercent": "change_percent", "symbol": "ticker"})
api_df.head()

,date,open,high,low,close,volume,change,change_percent,vwap,ticker
0,2021-02-26,25.150000,26.820000,24.290001,26.040001,621381.0,0.89,3.539,25.71667,TGH
1,2021-02-25,27.059999,27.530001,25.180000,25.260000,633600.0,-1.80,-6.652,25.99000,TGH
2,2021-02-24,28.000000,28.450001,26.770000,26.980000,1092300.0,-1.02,-3.643,27.40000,TGH
3,2021-02-23,27.400000,29.170000,25.500000,28.820000,1088500.0,1.42,5.182,27.83000,TGH
4,2021-02-22,25.030001,27.980000,24.990000,27.510000,1569000.0,2.48,9.908,26.82667,TGH


In [41]:
df = api_df.copy()

### 6. Check for duplicates

In [44]:
print("Duplicate value for Ticker and Date: {}".format(any(df.duplicated(subset=['date', 'ticker']))))


Duplicate value for Ticker and Date: False


### 7. Handling Missing Values

I created a function to check the percentage of missing values in each feature. deleing null values is always risky, it needs careful analysis. I attempt to check which feature has highest missing values an ddepending on it's important I will either delete or keep the features

In [45]:
# check for missing value
def percentMissing(df):
    
    df_numeric = df.select_dtypes(include=[np.number])
    numeric_cols = df_numeric.columns.values
    print(numeric_cols)
    
    # % of missing data
    for col in df.columns:
        # create missing indicator for features with missing data
        missing = df[col].isnull()
        pct_missing = np.mean(missing)*100
        print('{} - {}%'.format(col, round(pct_missing)))
        num_missing = np.sum(missing)
        
    return df

In [46]:
df_missing = percentMissing(df)

['open' 'high' 'low' 'close' 'volume' 'change' 'change_percent' 'vwap']
date - 0%
open - 0%
high - 0%
low - 0%
close - 0%
volume - 0%
change - 0%
change_percent - 0%
vwap - 0%
ticker - 0%


No missing value so far

In [47]:
df.to_csv('clean_api_data.csv', encoding='utf-8', index=False)

### Summary of the Milestone 4

I have performed the following steps:
1. Reading the data from API and Converting it into dataframe
2. Striped String data of whitespace and
3. Subsetting or Filtering dataset
4. Transforming datatype to correct format
5. Renaming data headers
6. Checked Duplicates
7. Identifying Missing value.

There was not much formatting on this api data, as most data was in correct format. All the missing symbols were already filtered out, some reason API was not fetching any data and I was getting error for them. 